# Parametrizing Panoptic Segmentation

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

import hydra
from src.utils import init_config, compute_panoptic_metrics, \
    grid_search_panoptic_partition, oracle_superpoint_clustering, \
    grid_search_panoptic_partition_FORinstance, panoptic_partition_FORinstance, \
    bayesian_optimize_panoptic_partition_FORinstance
import torch
from src.transforms import *
from src.utils.widgets import *
from src.data import *

# Very ugly fix to ignore lightning's warning messages about the
# trainer and modules not being connected
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = 'cuda'
# change for each experiment

experiment = 'forinstance_159lvlratio_2e-1voxel_update'
ckpt = '/home/valerio/Downloads/159lvlratio_2e-1voxel_update.ckpt'

stage = 'test'

In [3]:
# Parse the configs using hydra
cfg = init_config(overrides=[
    f"experiment=panoptic/{experiment}",
    f"ckpt_path={ckpt}"
])

datamodule = hydra.utils.instantiate(cfg.datamodule)
datamodule.prepare_data()
datamodule.setup()

if stage == 'val':
    dataset = datamodule.val_dataset
elif stage == 'train':
    dataset = datamodule.train_dataset
elif stage == 'test':
    dataset = datamodule.test_dataset
else:
    raise ValueError("Invalid stage")

dataset.print_classes()

model = hydra.utils.instantiate(cfg.model)

if ckpt is not None:
    model = model._load_from_checkpoint(cfg.ckpt_path)

model = model.eval().to(device)

Processing...


  0%|          | 0/21 [00:00<?, ?it/s]

Done!
Processing...


  0%|          | 0/11 [00:00<?, ?it/s]

Done!


0   Ground and low vegetation stuff
1   Tree                 thing
2   Unknown              void


In [ ]:
# 0, CULS
# 1 - 6, NIBIO
# 7, RMIT
# 8 - 9, SCION
# 10, TUWIEN

In [4]:
tiles = [0, 1, 7, 8, 10]
areas = ['CULS', 'NIBIO', 'RMIT', 'SCION', 'TUWIEN']
grid_search = True

for tile, area in zip(tiles, areas):
    print()
    print(f"TILE {tile}, AREA {area}")

    if grid_search:
        k_max = [10, 20]
        radius = [1, 3, 5]
        regularization = [10, 20]
        x_weight = [1e-2, 1e-1, 1]
        cutoff = [1000, 100]
    else:
        k_max = [20]
        radius = [3]
        regularization = [20]
        x_weight = [1e-1]
        cutoff = [1000]

    best_output, best_partitions, best_results, best_combination = grid_search_panoptic_partition_FORinstance(
        model,
        dataset,
        i_cloud=tile,
        graph_kwargs=dict(
            k_max=k_max,
            radius=radius),
        partition_kwargs=dict(
            regularization=regularization,
            x_weight=x_weight,
            cutoff=cutoff),
        mode='pas',
        verbose=True)



TILE 0, AREA CULS


Param Combinations: 100%|██████████| 72/72 [00:09<00:00,  7.55it/s]


    k_max  radius  regulari.  x_weight  cutoff     PQ     F1  TP  FP  FN
0      10       1         10      0.01    1000  88.43  81.08  15   2   5
1      10       1         10      0.01     100  84.80  73.17  15   6   5
2      10       1         10      0.10    1000  89.50  89.47  17   1   3
3      10       1         10      0.10     100  75.30  57.63  17  22   3
4      10       1         10      1.00    1000  87.68  86.96  20   6   0
..    ...     ...        ...       ...     ...    ...    ...  ..  ..  ..
67     20       5         20      0.01     100  85.55  74.29  13   2   7
68     20       5         20      0.10    1000  91.36  91.89  17   0   3
69     20       5         20      0.10     100  81.64  70.83  17  11   3
70     20       5         20      1.00    1000  87.99  85.00  17   3   3
71     20       5         20      1.00     100  73.16  50.75  17  30   3

[72 rows x 10 columns]

Best panoptic setup: 
Tree F1=93.02
   k_max  radius  regulari.  x_weight  cutoff
0     10       3 

Param Combinations: 100%|██████████| 72/72 [00:15<00:00,  4.78it/s]


    k_max  radius  regulari.  x_weight  cutoff     PQ     F1  TP   FP  FN
0      10       1         10      0.01    1000  75.78  64.41  19   10  11
1      10       1         10      0.01     100  75.40  63.77  22   17   8
2      10       1         10      0.10    1000  66.14  44.71  19   37  10
3      10       1         10      0.10     100  60.14  28.57  20   90  10
4      10       1         10      1.00    1000  53.57  17.70  10   73  20
..    ...     ...        ...       ...     ...    ...    ...  ..  ...  ..
67     20       5         20      0.01     100  49.47   0.00   0    0  27
68     20       5         20      0.10    1000  74.05  60.00  18   12  12
69     20       5         20      0.10     100  71.00  56.00  21   24   9
70     20       5         20      1.00    1000  58.31  26.26  13   57  16
71     20       5         20      1.00     100  55.27  16.04  17  165  13

[72 rows x 10 columns]

Best panoptic setup: 
Tree F1=67.80
   k_max  radius  regulari.  x_weight  cutoff
0    

...

    k_max  radius  regulari.  x_weight  cutoff     PQ     F1  TP  FP  FN
0      10       1         10      0.01    1000  43.68  19.35   9  20  55
1      10       1         10      0.01     100  48.79  33.33  19  31  45
2      10       1         10      0.10    1000  44.47  24.24  12  23  52
3      10       1         10      0.10     100  50.11  34.21  26  62  38
4      10       1         10      1.00    1000  33.52  17.14   9  32  55
..    ...     ...        ...       ...     ...    ...    ...  ..  ..  ..
67     20       5         20      0.01     100  42.87  16.00   6   5  58
68     20       5         20      0.10    1000  42.47  16.67   7  13  57
69     20       5         20      0.10     100  48.49  33.66  17  20  47
70     20       5         20      1.00    1000  41.43  13.79   6  17  58
71     20       5         20      1.00     100  49.88  34.90  26  59  38

[72 rows x 10 columns]

Best panoptic setup: 
Tree F1=37.68
   k_max  radius  regulari.  x_weight  cutoff
0     10       1 

Param Combinations: 100%|██████████| 72/72 [00:13<00:00,  5.25it/s]


    k_max  radius  regulari.  x_weight  cutoff     PQ     F1  TP   FP  FN
0      10       1         10      0.01    1000  67.08  46.81  11   11  14
1      10       1         10      0.01     100  63.06  34.48  10   23  15
2      10       1         10      0.10    1000  60.21  32.56  14   47  11
3      10       1         10      0.10     100  54.38  14.36  13  143  12
4      10       1         10      1.00    1000  52.97  14.81  10  100  15
..    ...     ...        ...       ...     ...    ...    ...  ..  ...  ..
67     20       5         20      0.01     100  56.10  17.65   3    6  22
68     20       5         20      0.10    1000  62.34  30.77   8   19  17
69     20       5         20      0.10     100  57.03  18.92   7   42  18
70     20       5         20      1.00    1000  58.32  28.87  14   58  11
71     20       5         20      1.00     100  52.67   9.96  14  242  11

[72 rows x 10 columns]

Best panoptic setup: 
Tree F1=46.81
   k_max  radius  regulari.  x_weight  cutoff
0    

Param Combinations: 100%|██████████| 72/72 [00:15<00:00,  4.61it/s]

    k_max  radius  regulari.  x_weight  cutoff     PQ     F1  TP   FP  FN
0      10       1         10      0.01    1000  38.94  13.51   5   34  30
1      10       1         10      0.01     100  39.31  14.43   7   55  28
2      10       1         10      0.10    1000  41.95  27.27  15   60  20
3      10       1         10      0.10     100  38.47  15.61  16  154  19
4      10       1         10      1.00    1000  36.87  13.25  10  106  25
..    ...     ...        ...       ...     ...    ...    ...  ..  ...  ..
67     20       5         20      0.01     100  35.91   7.69   2   15  33
68     20       5         20      0.10    1000  41.01  22.86   8   27  27
69     20       5         20      0.10     100  39.94  19.35   9   49  26
70     20       5         20      1.00    1000  40.83  23.85  13   61  22
71     20       5         20      1.00     100  36.38   9.59  13  223  22

[72 rows x 10 columns]

Best panoptic setup: 
Tree F1=30.38
   k_max  radius  regulari.  x_weight  cutoff
0    

In [ ]:
nag = dataset[tile]

# add the results to the nag for visualization
vox_y, vox_index, vox_obj_pred = best_output.voxel_panoptic_pred(super_index=nag[0].super_index)
nag[0].obj_pred = vox_obj_pred

center = nag[0].pos.mean(dim=0)[:2]
radius = 10
mask = torch.where(torch.linalg.norm(nag[0].pos[:, :2] - center, dim=1) < radius)[0]
nag_cylinder = nag.select(0, mask)
nag_cylinder.show(stuff_classes=[0], num_classes=2)